#### Importing Libraries and Dataset

In [1]:
import numpy as np
from statistics import mean
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pandas as pd
import seaborn as sns
from tslearn.clustering import TimeSeriesKMeans
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, SimpleRNN 
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, Dropout
import tensorflow as tf
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow import keras
import seaborn as sns
import pickle
import functools
import fastdtw
import scipy.spatial.distance as sd
from tslearn.clustering import silhouette_score
#from datetime import datetime
consumption_df = pd.read_csv('Final Datasets/energy_consumption.csv')
consumption_df

2022-05-19 12:25:38.405475: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-19 12:25:38.405500: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


,Unnamed: 0,timestamp,energy_kWh,House Number,electronics workbench energy,network equipment energy,home office energy,RAE home office energy,RAE Mobile Phone Charger Energy
0,0,2012-06-01 01:00:00,1.0110,1,0.008686,0.019316,0.018232,0.134570,0.000431
1,1,2012-06-01 02:00:00,0.4510,1,0.007178,0.019338,0.013745,0.061839,0.000215
2,2,2012-06-01 03:00:00,0.5050,1,0.007324,0.019336,0.014177,0.068852,0.000236
3,3,2012-06-01 04:00:00,0.4410,1,0.007151,0.019339,0.013664,0.060540,0.000211
4,4,2012-06-01 05:00:00,0.4680,1,0.007224,0.019338,0.013881,0.064047,0.000221
...,...,...,...,...,...,...,...,...,...
611013,123761,2020-05-19 19:00:00,3.5802,28,0.028408,0.038470,0.069307,0.801381,0.002441
611014,123762,2020-05-19 20:00:00,3.4398,28,0.027762,0.038479,0.067384,0.770210,0.002348
611015,123763,2020-05-19 21:00:00,2.3049,28,0.022537,0.038556,0.051836,0.518247,0.001601
611016,123764,2020-05-19 22:00:00,2.4921,28,0.023399,0.038544,0.054401,0.559808,0.001724


In [2]:
if 'Unnamed: 0' in consumption_df.columns:
    consumption_df.drop('Unnamed: 0',axis=1, inplace = True)

In [3]:
consumption_df

,timestamp,energy_kWh,House Number,electronics workbench energy,network equipment energy,home office energy,RAE home office energy,RAE Mobile Phone Charger Energy
0,2012-06-01 01:00:00,1.0110,1,0.008686,0.019316,0.018232,0.134570,0.000431
1,2012-06-01 02:00:00,0.4510,1,0.007178,0.019338,0.013745,0.061839,0.000215
2,2012-06-01 03:00:00,0.5050,1,0.007324,0.019336,0.014177,0.068852,0.000236
3,2012-06-01 04:00:00,0.4410,1,0.007151,0.019339,0.013664,0.060540,0.000211
4,2012-06-01 05:00:00,0.4680,1,0.007224,0.019338,0.013881,0.064047,0.000221
...,...,...,...,...,...,...,...,...
611013,2020-05-19 19:00:00,3.5802,28,0.028408,0.038470,0.069307,0.801381,0.002441
611014,2020-05-19 20:00:00,3.4398,28,0.027762,0.038479,0.067384,0.770210,0.002348
611015,2020-05-19 21:00:00,2.3049,28,0.022537,0.038556,0.051836,0.518247,0.001601
611016,2020-05-19 22:00:00,2.4921,28,0.023399,0.038544,0.054401,0.559808,0.001724


#### Converting Dataset to House-wise dictionary

In [4]:
house_wise_consumption = dict()
for house_number in consumption_df['House Number'].unique():
    house_data = consumption_df[consumption_df['House Number'] == house_number]
    house_wise_consumption[house_number] = house_data.copy()
house_wise_consumption[1]
# house_wise_consumption[14]['timestamp'].iloc[0]
# house_wise_consumption[14]['timestamp'].iloc[-1]
# house_wise_consumption[14]['House Number'].iloc[0]

,timestamp,energy_kWh,House Number,electronics workbench energy,network equipment energy,home office energy,RAE home office energy,RAE Mobile Phone Charger Energy
0,2012-06-01 01:00:00,1.011,1,0.008686,0.019316,0.018232,0.134570,0.000431
1,2012-06-01 02:00:00,0.451,1,0.007178,0.019338,0.013745,0.061839,0.000215
2,2012-06-01 03:00:00,0.505,1,0.007324,0.019336,0.014177,0.068852,0.000236
3,2012-06-01 04:00:00,0.441,1,0.007151,0.019339,0.013664,0.060540,0.000211
4,2012-06-01 05:00:00,0.468,1,0.007224,0.019338,0.013881,0.064047,0.000221
...,...,...,...,...,...,...,...,...
29226,2015-10-02 20:00:00,2.101,1,0.011621,0.019273,0.026968,0.276137,0.000851
29227,2015-10-02 21:00:00,1.646,1,0.010396,0.019291,0.023322,0.217043,0.000675
29228,2015-10-02 22:00:00,1.319,1,0.009516,0.019304,0.020701,0.174573,0.000549
29229,2015-10-02 23:00:00,1.473,1,0.009930,0.019298,0.021935,0.194574,0.000609


In [5]:
# TODO: Print the usage pattern in form of a graph
# Each cluster represents one line in the time series graph
# Plot december consumption (2017) values for each house belonging to the same cluster in a single graph
# Also plot for the whole year (annual 2017) for each house belonging to the same cluster in a single graph

#### Comparing Timestamps for clustering

In [6]:
timestamp_comparison = list()
for house in house_wise_consumption:
    first_timestamp = house_wise_consumption[house]['timestamp'].iloc[0]
    last_timestamp = house_wise_consumption[house]['timestamp'].iloc[-1]
    timestamp_comparison.append([house, first_timestamp, last_timestamp])
timestamp_comparison
timestamp_comparison_df = pd.DataFrame(timestamp_comparison, columns = ['House', 'First Timestamp', 'Last Timestamp'])
timestamp_comparison_df

,House,First Timestamp,Last Timestamp
0,1,2012-06-01 01:00:00,2015-10-03 00:00:00
1,2,2016-06-09 00:00:00,2019-11-20 23:00:00
2,3,2015-01-27 00:00:00,2018-01-29 23:00:00
3,4,2015-01-30 00:00:00,2018-01-29 23:00:00
4,5,2015-01-30 00:00:00,2018-01-29 23:00:00
5,6,2015-01-30 00:00:00,2018-01-29 23:00:00
6,7,2015-05-30 00:00:00,2018-01-29 23:00:00
7,8,2015-02-21 00:00:00,2018-02-20 23:00:00
8,9,2015-05-01 00:00:00,2018-02-21 23:00:00
9,10,2015-02-21 00:00:00,2018-02-20 23:00:00


In [7]:
houses_with_ts = []
sample_timestamp = '2017-07-27 00:00:00' #2017-07-27 00:00:00
for house in house_wise_consumption:
    if sample_timestamp in house_wise_consumption[house]['timestamp'].unique():
        houses_with_ts.append(house)
len(houses_with_ts)
#house_wise_consumption[8][house_wise_consumption[8]['timestamp'] >= '2017-11-17'].head(60)

21

#### Creating a December consumption dictionary of dataframes

In [8]:
december_consumption = dict()
for house in house_wise_consumption:
    house_december_consumption = house_wise_consumption[house][(house_wise_consumption[house]['timestamp'] <= '2017-11-25') & (house_wise_consumption[house]['timestamp'] >= '2017-07-27')][['timestamp', 'energy_kWh']]
    december_consumption[house] = house_december_consumption.copy()
    december_consumption[house].drop_duplicates(subset='timestamp', keep="last", inplace = True)
def calculate_rows():
    december_rows = []
    for house in december_consumption:
        december_rows.append([house, len(december_consumption[house])])
    december_rows
    december_rows_df = pd.DataFrame(december_rows, columns = ['House', 'Rows'])
    return december_rows_df
def calculate_incomplete_rows():
    december_rows_df = calculate_rows()
    december_rows_incomplete_df = december_rows_df[(december_rows_df['Rows'] > 0) & (december_rows_df['Rows'] < 2904)]
    return december_rows_incomplete_df
def fill_na_december_consumption(complete_df, incomplete_df):
    complete_df = complete_df.copy()
    year_incomplete = pd.Timestamp(incomplete_df.iloc[0]['timestamp']).year
    year_complete = pd.Timestamp(complete_df.iloc[0]['timestamp']).year
    complete_df['timestamp'] = complete_df['timestamp'].str.replace(str(year_complete), str(year_incomplete))
    fill_missing_values = complete_df[~complete_df.timestamp.isin(incomplete_df.timestamp)].copy()
    #print(fill_missing_values)
    fill_columns = ['energy_kWh']
    for fill_column in fill_columns:
        fill_missing_values[fill_column] = incomplete_df[fill_column].mean()
    incomplete_df = pd.concat([incomplete_df, fill_missing_values])
    incomplete_df = incomplete_df.sort_values(by=['timestamp'])
    return incomplete_df
def calculate_missing_december_rows():
    december_rows_df = calculate_rows()
    december_rows_missing_df = december_rows_df[(december_rows_df['Rows'] == 0)]
    return december_rows_missing_df


In [9]:
comparison_df =  pd.DataFrame()
for house in december_consumption:
    consumption = december_consumption[house][['timestamp', 'energy_kWh']]
    consumption.rename(columns={'energy_kWh':house}, inplace=True)
    # consumption.set_index('timestamp', inplace = True)
    if not consumption.empty:
        if comparison_df.empty:
            comparison_df = consumption
        else:
            comparison_df=(pd.merge(comparison_df, consumption, how='outer', on='timestamp'))
consumption.set_index('timestamp', inplace = True)
comparison_df['energy_kWh'] = comparison_df.mean(axis=1)
comparison_df = comparison_df[['timestamp', 'energy_kWh']]
comparison_df

/tmp/ipykernel_40780/236779599.py:12: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  comparison_df['energy_kWh'] = comparison_df.mean(axis=1)


,timestamp,energy_kWh
0,2017-07-27 00:00:00,0.392857
1,2017-07-27 01:00:00,0.367619
2,2017-07-27 02:00:00,0.360952
3,2017-07-27 03:00:00,0.370476
4,2017-07-27 04:00:00,0.341429
...,...,...
2899,2017-11-24 23:00:00,0.658696
2900,2017-08-12 03:00:00,0.360000
2901,2017-08-17 20:00:00,0.857000
2902,2017-09-05 15:00:00,0.780000


In [10]:
# calculate_rows()

In [11]:
for house in list(calculate_incomplete_rows()['House']):
    # TODO - Replace House 10 with mean/median
    december_consumption[house] = fill_na_december_consumption(comparison_df, december_consumption[house])
# for house in list(calculate_incomplete_rows()['House']):
#     # TODO - Replace House 10 with mean/median
#     december_consumption[house] = fill_na_december_consumption(december_consumption[2], december_consumption[house])
calculate_rows()

,House,Rows
0,1,0
1,2,2904
2,3,2904
3,4,2904
4,5,2904
5,6,2904
6,7,2904
7,8,2904
8,9,2904
9,10,2904


##### Houses with no December-17 consumption values

In [12]:
timestamp_comparison_df.set_index('House').join(calculate_missing_december_rows().set_index('House'), how='inner')

,First Timestamp,Last Timestamp,Rows
House,,,
1,2012-06-01 01:00:00,2015-10-03 00:00:00,0
17,2016-06-01 00:00:00,2017-04-29 23:00:00,0
26,2019-11-29 00:00:00,2020-05-20 23:00:00,0
27,2019-03-06 00:00:00,2020-05-20 23:00:00,0
28,2018-08-01 00:00:00,2020-05-19 23:00:00,0


In [13]:
# missing_december_houses_dates = {
#     1: ['2014-11-25', '2014-11-01'],
#     17: ['2016-11-25', '2016-11-01'],
#     26: ['2019-11-25', '2019-11-01'],
#     27: ['2019-11-25', '2019-11-01'],
#     28: ['2019-11-25', '2019-11-01'],
# }

In [14]:
# for house in missing_december_houses_dates:
#     house_december_consumption = house_wise_consumption[house][(house_wise_consumption[house]['timestamp'] <= missing_december_houses_dates[house][0]) & (house_wise_consumption[house]['timestamp'] >= missing_december_houses_dates[house][1])]
#     #print((house, missing_december_houses_dates[house][0]))
#     december_consumption[house] = house_december_consumption.copy()
#     december_consumption[house].drop_duplicates(subset='timestamp', keep="last", inplace = True)

##### Filling missing values for December consumption

In [15]:
# calculate_incomplete_rows()

In [16]:
# for house in list(calculate_incomplete_rows()['House']):
#     december_consumption[house] = fill_na_december_consumption(december_consumption[10], december_consumption[house])

In [17]:
#calculate_rows()december_consumption
del december_consumption[1]
del december_consumption[17]
del december_consumption[26]
del december_consumption[27]
del december_consumption[28]
del december_consumption[21]
del december_consumption[16]
for house in december_consumption:
    december_consumption[house].drop_duplicates(subset='timestamp', keep="last", inplace = True)
calculate_rows()

,House,Rows
0,2,2904
1,3,2904
2,4,2904
3,5,2904
4,6,2904
5,7,2904
6,8,2904
7,9,2904
8,10,2904
9,11,2904


#### Houses to be clustered

In [18]:
print('Houses to be clustered: ')
clustered_df = calculate_rows()[calculate_rows()['Rows'] ==  2904]
clustered_df

Houses to be clustered: 


,House,Rows
0,2,2904
1,3,2904
2,4,2904
3,5,2904
4,6,2904
5,7,2904
6,8,2904
7,9,2904
8,10,2904
9,11,2904


#### Converting to clustering matrix

In [19]:
clustering_matrix = []
for house in calculate_rows()[calculate_rows()['Rows'] ==  2904]['House']:
    clustering_matrix.append(np.array(december_consumption[house]['energy_kWh']).reshape(-1,1))
clustering_matrix = np.array(clustering_matrix)
# For 24 days
clustering_matrix.shape

(21, 2904, 1)

#### Time series clustering of all households except house 26

In [20]:
# km_sdtw = TimeSeriesKMeans(n_clusters=5, metric="softdtw", max_iter=1,
#                             max_iter_barycenter=5,
#                             metric_params={"gamma": .5},
#                             random_state=0).fit_predict(clustering_matrix)

In [21]:
# km_sdtw

In [ ]:
silhouette = []
for i in range(2,10):
    km = TimeSeriesKMeans(n_clusters=i, metric="softdtw", max_iter=1,
                            max_iter_barycenter=5,
                            metric_params={"gamma": .5},
                            random_state=0)
    km.fit(clustering_matrix)
    silhouette.append(silhouette_score(clustering_matrix,
                           km.predict(clustering_matrix),
                           metric="softdtw"))

In [ ]:
clustered_df['Cluster'] = km.predict(clustering_matrix)
clustered_df = clustered_df[['House', 'Cluster']].set_index('House')
clustered_df

In [ ]:
# Elbow graph 
elbow_x = list(range(2,10))
elbow_y = silhouette
plt.plot(elbow_x,elbow_y)
plt.plot(5,0.052, marker=6, label='Optimal value')

plt.xlabel("N-value for clustering")  # add X-axis label
plt.ylabel("Silhoutte score")  # add Y-axis label
plt.title("K-means clustering optimization")  # add title
plt.legend()

In [ ]:
# clustered_df['Cluster'] = km_sdtw

In [ ]:
#clustered_df = clustered_df[['House', 'Cluster']].set_index('House')
# clustered_df[clustered_df['Cluster'] == 3]

In [ ]:
# #clustered_df = pd.read_csv('Tables/clustered.csv')
# clustered_df.to_csv('Tables/clustered.csv')

# https://github.com/tslearn-team/tslearn/issues/278
# https://stats.stackexchange.com/questions/478618/silhouette-score-not-robust-when-clustering-time-series-with-tslearn
# https://tslearn.readthedocs.io/en/stable/gen_modules/clustering/tslearn.clustering.silhouette_score.html